In [ ]:
import dautil as dl
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML

In [ ]:
temp = dl.data.Weather.load()['TEMP'].dropna()
monthly = temp.resample('M')

In [ ]:
%matplotlib inline
context = dl.nb.Context('analyzing_peaks')
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 2, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 2, context)
max_locs = signal.argrelmax(monthly.values)
sp.ax.plot(monthly.index, monthly, label='Monthly means')
sp.ax.plot(monthly.index[max_locs], monthly.values[max_locs], 
           'o', label='Tops')
sp.label(ylabel_params=dl.data.Weather.get_header('TEMP'))

annual_max = dl.ts.groupby_year(temp).max()
sp.next_ax().plot(annual_max.index, annual_max, label='Annual Maximum Series')
dl.plotting.plot_polyfit(sp.ax, annual_max.index, annual_max.values)
sp.label(ylabel_params=dl.data.Weather.get_header('TEMP'))

_, threshhold = dl.stats.outliers(temp, method='percentiles')
over_threshhold = temp > threshhold
streaks = dl.ts.groupby_year(over_threshhold).apply(
    lambda x: dl.collect.longest_streak(x, 1))
sp.next_ax().plot(streaks.index, streaks)
dl.plotting.plot_polyfit(sp.ax, streaks.index, streaks.values)
over_threshhold = dl.ts.groupby_year(over_threshhold).mean()
sp.label()

sp.label(advance=True)
sns.distplot(annual_max, ax=sp.ax)
sp.label(xlabel_params=dl.data.Weather.get_header('TEMP'))
HTML(sp.exit())